In [3]:
import re

In [2]:
def length_standard(n):
    out = []
    for i in range(n):
        if i % 10 == 0:
            out.append((i - 100*(i/100.)) / 10.)
        elif i % 5 == 0:
            out.append(":")
        else:
            out.append(".")
    return "".join(map(str, out))

# Fast 5 data reading routines 

In [4]:
def get_file_and_channel(filename):
    result = re.search(r'ch(\d+)_file(\d+)_', filename)
    file_id = int(result.group(2))
    channel_id = int(result.group(1))
    return file_id, channel_id

####Events-Table
file	strand	mean	start	stdv	length	model_state	model_level	move	p_model_state	mp_model_state	p_mp_model_state	p_A	p_C	p_G	p_T	raw_index

only look at the "template" seq for the beginning


In [ ]:
def process_metrichor_files(files, verbose=False):
    """
    reads every filename in files with poretools and 
    extracts events and metadata
    """
    
    file_data = []
    for file in files: 
        f_obj = {}
        file_id, ch_id = get_file_and_channel(file)
        if (verbose): 
            print("processing file {0} channel {1}".format(
                file_id, ch_id))
        f_obj["channel"] = ch_id
        f_obj["file_id"] = file_id

        fastq = !poretools fastq {file} | grep template -A3
        if not fastq: 
            print("Empty file: {0}. Skipped.".format(file))
            continue
       
        f_obj["called_seq"] = fastq[1]
        f_obj["called_qual"] = fastq[3]

        events = !poretools events {file} | grep template
        events = [line.split("\t") for line in events]

        file_events = []
        for ev in events: 
            r_ev = dict()
            r_ev["start"] = round(float(ev[3]) * SRATE)
            r_ev["end"] = round(r_ev["start"] + float(ev[5]) * SRATE)
            r_ev["mp_kmer"] = ev[10] #maximum posterior
            r_ev["kmer"] = ev[6] #model state
            r_ev["move"] = int(ev[8])
            file_events.append(r_ev)

        ## check if raw data is available
        file_id, ch_id = get_file_and_channel(file)

        f_obj["channel"] = ch_id
        f_obj["file_id"] = file_id
        f_obj["events"] = file_events
        file_data.append(f_obj)
        
    return file_data
    
    